In [2]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "natasha pritykovskaya Custom Estimator") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [4]:
spark

In [5]:
from pyspark.ml import Estimator

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.datasets import make_classification

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X, y = make_classification(random_state=5757)

In [10]:
y.shape

(100,)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5757)

In [12]:
X_train.shape

(80, 20)

In [13]:
est = LogisticRegression(random_state=5757)

In [14]:
est.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=5757, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
from pyspark.ml.linalg import DenseVector

In [16]:
from pyspark.sql.types import StructType, IntegerType, StructField, FloatType, ArrayType
from pyspark.ml.linalg import VectorUDT

In [17]:
type(X_test)

numpy.ndarray

In [18]:
DenseVector(X_test[0])

DenseVector([0.1671, 2.2893, 2.2519, -0.3188, -1.2869, -0.5163, 0.8432, -1.6526, 1.4384, -0.1379, -0.2343, 0.4912, -0.2036, -1.2666, 1.1291, -0.0674, 2.6995, 0.87, 0.3678, 0.1223])

## Способ №1, правильный, но не работает :(

In [19]:
schema = StructType(fields=[
    StructField("features", VectorUDT()),
    StructField("label", IntegerType())
])

In [20]:
df_test = spark.createDataFrame(zip(map(DenseVector, X_test), map(int, y_test)), schema=schema)

In [21]:
df_test.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.16713128673558...|    0|
|[2.50158633810562...|    0|
|[-0.7699917522248...|    0|
|[-0.9514898817744...|    0|
|[1.32598926517448...|    0|
|[-0.5091954518113...|    0|
|[-0.9442948966018...|    1|
|[0.53537784380006...|    1|
|[-1.9316295195085...|    0|
|[-0.8788162423442...|    1|
|[0.47792785291472...|    1|
|[-0.7537969085613...|    1|
|[-0.1103342853056...|    0|
|[0.21903394455633...|    1|
|[-0.7010314631582...|    1|
|[-0.4024860125426...|    1|
|[1.91508615582208...|    1|
|[1.03957987692160...|    1|
|[-1.9962336859493...|    1|
|[0.03293939843484...|    1|
+--------------------+-----+



In [22]:
est_broadcast = spark.sparkContext.broadcast(est)

In [23]:
import pyspark.sql.functions as f

In [24]:
@f.pandas_udf(FloatType())
def predict(series):
    predictions = est_broadcast.value.predict(series)
    return pd.Series(predictions)

In [25]:
df_test.withColumn("prediction", predict("features")).show()

Py4JJavaError: An error occurred while calling o112.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 8, bd-node4.newprolab.com, executor 1): java.lang.UnsupportedOperationException: Unsupported data type: struct<type:tinyint,size:int,indices:array<int>,values:array<double>>
	at org.apache.spark.sql.execution.arrow.ArrowUtils$.toArrowType(ArrowUtils.scala:56)
	at org.apache.spark.sql.execution.arrow.ArrowUtils$.toArrowField(ArrowUtils.scala:92)
	at org.apache.spark.sql.execution.arrow.ArrowUtils$$anonfun$toArrowSchema$1.apply(ArrowUtils.scala:116)
	at org.apache.spark.sql.execution.arrow.ArrowUtils$$anonfun$toArrowSchema$1.apply(ArrowUtils.scala:115)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.Iterator$class.foreach(Iterato

Все это происходит из-за этого https://issues.apache.org/jira/browse/SPARK-19653

## Способ №2, неправильный, но работает :/

In [26]:
import pandas as pd

In [27]:
@f.udf(ArrayType(FloatType()))
def vectorToArray(row):
    return DenseVector(row).tolist()

df_test = df_test.withColumn("features_array", vectorToArray("features"))

In [28]:
df_test.show()

+--------------------+-----+--------------------+
|            features|label|      features_array|
+--------------------+-----+--------------------+
|[0.16713128673558...|    0|[0.16713129, 2.28...|
|[2.50158633810562...|    0|[2.5015864, 1.394...|
|[-0.7699917522248...|    0|[-0.76999176, 0.1...|
|[-0.9514898817744...|    0|[-0.95148987, 1.6...|
|[1.32598926517448...|    0|[1.3259892, 1.287...|
|[-0.5091954518113...|    0|[-0.50919545, 2.0...|
|[-0.9442948966018...|    1|[-0.94429487, 1.2...|
|[0.53537784380006...|    1|[0.53537786, 0.08...|
|[-1.9316295195085...|    0|[-1.9316295, -1.3...|
|[-0.8788162423442...|    1|[-0.87881625, -0....|
|[0.47792785291472...|    1|[0.47792786, 1.28...|
|[-0.7537969085613...|    1|[-0.75379694, 1.9...|
|[-0.1103342853056...|    0|[-0.110334285, 3....|
|[0.21903394455633...|    1|[0.21903394, -1.2...|
|[-0.7010314631582...|    1|[-0.70103145, 0.6...|
|[-0.4024860125426...|    1|[-0.40248603, -1....|
|[1.91508615582208...|    1|[1.9150862, -1.07...|


In [29]:
type(df_test.take(1)[0][0])

pyspark.ml.linalg.DenseVector

In [30]:
type(df_test.take(1)[0][2])

list

In [40]:
@f.pandas_udf(FloatType())
def predict(series):
    # Необходимо сделать преобразование, потому что на вход приходит pd.Series((list1, list2, ...))
    predictions = est_broadcast.value.predict(series.tolist())
    return pd.Series(predictions)

In [34]:
df_test.withColumn("prediction", predict("features_array")).show()

+--------------------+-----+--------------------+----------+
|            features|label|      features_array|prediction|
+--------------------+-----+--------------------+----------+
|[0.16713128673558...|    0|[0.16713129, 2.28...|       0.0|
|[2.50158633810562...|    0|[2.5015864, 1.394...|       0.0|
|[-0.7699917522248...|    0|[-0.76999176, 0.1...|       0.0|
|[-0.9514898817744...|    0|[-0.95148987, 1.6...|       0.0|
|[1.32598926517448...|    0|[1.3259892, 1.287...|       0.0|
|[-0.5091954518113...|    0|[-0.50919545, 2.0...|       0.0|
|[-0.9442948966018...|    1|[-0.94429487, 1.2...|       1.0|
|[0.53537784380006...|    1|[0.53537786, 0.08...|       1.0|
|[-1.9316295195085...|    0|[-1.9316295, -1.3...|       0.0|
|[-0.8788162423442...|    1|[-0.87881625, -0....|       1.0|
|[0.47792785291472...|    1|[0.47792786, 1.28...|       1.0|
|[-0.7537969085613...|    1|[-0.75379694, 1.9...|       1.0|
|[-0.1103342853056...|    0|[-0.110334285, 3....|       1.0|
|[0.21903394455633...|  

In [55]:
import pickle

In [56]:
with open("logistic_model.pk", "wb") as f:
    pickle.dump(est, f)

In [57]:
from pyspark import keyword_only
from pyspark.ml import Model, Estimator
from pyspark.ml.param import Param, Params, TypeConverters
from pyspark.ml.param.shared import HasFeaturesCol, HasLabelCol, HasPredictionCol

[Model base class](https://github.com/apache/spark/blob/v2.4.0/python/pyspark/ml/base.py#L243)


In [58]:
class SKLogisticRegreesionModel(Model, HasFeaturesCol, HasLabelCol, HasPredictionCol):
    model_file = Param(Params._dummy(), "model_file",
                      "path to pickled scikit-learn logistic regression model",
                      typeConverter=TypeConverters.toString)
    @keyword_only
    def __init__(self, model_file=None, featuresCol="features", labelCol="label", predictionCol="prediction"):
        super(SKLogisticRegreesionModel, self).__init__()
        
        if model_file is None:
            raise ValueError("model_file must be specified!")
            
        with open(model_file, "rb") as f:
            self.estimator = pickle.load(f)
            
        kwargs = self._input_kwargs
        self._set(**kwargs)
        
    def _transform(self, dataset):
        return dataset.withColumn(self.getPredictionCol(), predict(self.getFeaturesCol()))

In [59]:
spark_est = SKLogisticRegreesionModel(model_file="logistic_model.pk", featuresCol="features_array")

In [61]:
spark_est.transform(df_test).show()

+--------------------+-----+--------------------+----------+
|            features|label|      features_array|prediction|
+--------------------+-----+--------------------+----------+
|[0.16713128673558...|    0|[0.16713129, 2.28...|       0.0|
|[2.50158633810562...|    0|[2.5015864, 1.394...|       0.0|
|[-0.7699917522248...|    0|[-0.76999176, 0.1...|       0.0|
|[-0.9514898817744...|    0|[-0.95148987, 1.6...|       0.0|
|[1.32598926517448...|    0|[1.3259892, 1.287...|       0.0|
|[-0.5091954518113...|    0|[-0.50919545, 2.0...|       0.0|
|[-0.9442948966018...|    1|[-0.94429487, 1.2...|       1.0|
|[0.53537784380006...|    1|[0.53537786, 0.08...|       1.0|
|[-1.9316295195085...|    0|[-1.9316295, -1.3...|       0.0|
|[-0.8788162423442...|    1|[-0.87881625, -0....|       1.0|
|[0.47792785291472...|    1|[0.47792786, 1.28...|       1.0|
|[-0.7537969085613...|    1|[-0.75379694, 1.9...|       1.0|
|[-0.1103342853056...|    0|[-0.110334285, 3....|       1.0|
|[0.21903394455633...|  

In [62]:
from pyspark.ml import Pipeline

In [63]:
pipeline = Pipeline(stages=[
    spark_est
])

In [64]:
pipeline_model = pipeline.fit(df_test)

In [65]:
pipeline_model.transform(df_test).show()

+--------------------+-----+--------------------+----------+
|            features|label|      features_array|prediction|
+--------------------+-----+--------------------+----------+
|[0.16713128673558...|    0|[0.16713129, 2.28...|       0.0|
|[2.50158633810562...|    0|[2.5015864, 1.394...|       0.0|
|[-0.7699917522248...|    0|[-0.76999176, 0.1...|       0.0|
|[-0.9514898817744...|    0|[-0.95148987, 1.6...|       0.0|
|[1.32598926517448...|    0|[1.3259892, 1.287...|       0.0|
|[-0.5091954518113...|    0|[-0.50919545, 2.0...|       0.0|
|[-0.9442948966018...|    1|[-0.94429487, 1.2...|       1.0|
|[0.53537784380006...|    1|[0.53537786, 0.08...|       1.0|
|[-1.9316295195085...|    0|[-1.9316295, -1.3...|       0.0|
|[-0.8788162423442...|    1|[-0.87881625, -0....|       1.0|
|[0.47792785291472...|    1|[0.47792786, 1.28...|       1.0|
|[-0.7537969085613...|    1|[-0.75379694, 1.9...|       1.0|
|[-0.1103342853056...|    0|[-0.110334285, 3....|       1.0|
|[0.21903394455633...|  

[Estimator base class](https://github.com/apache/spark/blob/v2.4.0/python/pyspark/ml/base.py#L243)


In [67]:
class SKLogisticRegression(Estimator, HasFeaturesCol, HasPredictionCol, HasLabelCol):
    @keyword_only
    def __init__(self, featuresCol="features", predictionCol="prediction", labelCol="label"):
        super(SKLogisticRegression, self).__init__()
        kwargs = self._input_kwargs
        self._set(**kwargs)
        
    def _fit(self, dataset):
        local_dataset = dataset.select(self.getFeaturesCol(), self.getLabelCol()).toPandas()
        self.est = LogisticRegression()
        self.est.fit(local_dataset[self.getFeaturesCol()].tolist(), local_dataset[self.getLabelCol()])
        self.model_file = "logistic_regression.pk"
        with open(self.model_file, "wb") as f:
            pickle.dump(self.est, f)
        return SKLogisticRegreesionModel(model_file=self.model_file, predictionCol=self.getPredictionCol(),
                                         featuresCol=self.getFeaturesCol(), labelCol=self.getLabelCol())

In [68]:
spark_est = SKLogisticRegression(featuresCol="features_array")

In [69]:
spark_est_model = spark_est.fit(df_test)

In [70]:
spark_est_model

SKLogisticRegreesionModel_8d8d7bdab78e

In [71]:
spark_est_model.transform(df_test).show()

+--------------------+-----+--------------------+----------+
|            features|label|      features_array|prediction|
+--------------------+-----+--------------------+----------+
|[0.16713128673558...|    0|[0.16713129, 2.28...|       0.0|
|[2.50158633810562...|    0|[2.5015864, 1.394...|       0.0|
|[-0.7699917522248...|    0|[-0.76999176, 0.1...|       0.0|
|[-0.9514898817744...|    0|[-0.95148987, 1.6...|       0.0|
|[1.32598926517448...|    0|[1.3259892, 1.287...|       0.0|
|[-0.5091954518113...|    0|[-0.50919545, 2.0...|       0.0|
|[-0.9442948966018...|    1|[-0.94429487, 1.2...|       1.0|
|[0.53537784380006...|    1|[0.53537786, 0.08...|       1.0|
|[-1.9316295195085...|    0|[-1.9316295, -1.3...|       0.0|
|[-0.8788162423442...|    1|[-0.87881625, -0....|       1.0|
|[0.47792785291472...|    1|[0.47792786, 1.28...|       1.0|
|[-0.7537969085613...|    1|[-0.75379694, 1.9...|       1.0|
|[-0.1103342853056...|    0|[-0.110334285, 3....|       1.0|
|[0.21903394455633...|  

In [73]:
pipeline = Pipeline(stages=[
    spark_est
])

In [74]:
pipeline_model = pipeline.fit(df_test)

In [75]:
pipeline_model.transform(df_test).show()

+--------------------+-----+--------------------+----------+
|            features|label|      features_array|prediction|
+--------------------+-----+--------------------+----------+
|[0.16713128673558...|    0|[0.16713129, 2.28...|       0.0|
|[2.50158633810562...|    0|[2.5015864, 1.394...|       0.0|
|[-0.7699917522248...|    0|[-0.76999176, 0.1...|       0.0|
|[-0.9514898817744...|    0|[-0.95148987, 1.6...|       0.0|
|[1.32598926517448...|    0|[1.3259892, 1.287...|       0.0|
|[-0.5091954518113...|    0|[-0.50919545, 2.0...|       0.0|
|[-0.9442948966018...|    1|[-0.94429487, 1.2...|       1.0|
|[0.53537784380006...|    1|[0.53537786, 0.08...|       1.0|
|[-1.9316295195085...|    0|[-1.9316295, -1.3...|       0.0|
|[-0.8788162423442...|    1|[-0.87881625, -0....|       1.0|
|[0.47792785291472...|    1|[0.47792786, 1.28...|       1.0|
|[-0.7537969085613...|    1|[-0.75379694, 1.9...|       1.0|
|[-0.1103342853056...|    0|[-0.110334285, 3....|       1.0|
|[0.21903394455633...|  

In [76]:
spark.stop()